In [1]:
path_data = '/data'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")
import os

import pandas as pd

import torch
from pkg_resources import packaging
from PIL import Image
import tqdm
from pathlib import Path
print("Torch version:", torch.__version__)

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

plt.style.use('ggplot')

Torch version: 2.2.1+cu121


In [3]:
data_unfiltered = pd.read_csv("/".join([path_data, "images_v2.csv"]))
#display(data_unfiltered)

In [4]:
len(data_unfiltered)

38479

### Remove subjective topics

In [5]:
subjective_topics = ['Favorite home decorations', 'Favourite item in kitchen', 'Favourite sports clubs', 'How the most loved item is used', 'icons', 'Idols', 'Latest furniture bought', ' looking over the shoulder', 'Most loved item', 'Most loved toy', 'Most played songs on the radio', 'Music idol', 'Next big thing you are planning to buy', 'Playing with most loved toy', 'Thing I dream about having', 'Things I wish I had', 'Using most loved item', 'Youth culture', 'What I wish I could buy']

In [6]:
len(subjective_topics)

19

In [7]:
unique_topics = data_unfiltered['topics'].unique()

In [8]:
Subjective_topic_list = []

for topic in unique_topics:
    topic_ = topic.replace('/ ', ', ')
    topic_split = topic_.split(',')
    for split in topic_split:
        if split in subjective_topics:
            Subjective_topic_list.append(topic)

In [9]:
data = data_unfiltered[~data_unfiltered['topics'].isin(Subjective_topic_list)]
display(data)

,id,country.name,country.id,region.id,type,imageRelPath,topics,place,income
1,5d4bf31ccf0b3a0f3f359814,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359814/5d4bf31ccf0b3a...,Family snapshots,butoyi,26.994581
2,5d4bf31ccf0b3a0f3f35982a,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982a/5d4bf31ccf0b3a...,Cutlery,butoyi,26.994581
3,5d4bf31ccf0b3a0f3f35982e,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982e/5d4bf31ccf0b3a...,Family,butoyi,26.994581
4,5d4bf31ccf0b3a0f3f359830,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359830/5d4bf31ccf0b3a...,Place where eating dinner,butoyi,26.994581
5,5d4bf31dcf0b3a0f3f35983c,Burundi,bi,af,image,assets/5d4bf31dcf0b3a0f3f35983c/5d4bf31dcf0b3a...,Plate of food,butoyi,26.994581
...,...,...,...,...,...,...,...,...,...
38474,5ec4f5513f62767d97a47324,France,fr,eu,image,assets/5ec4f5513f62767d97a47324/5ec4f5513f6276...,Bed,larriere,19671.000000
38475,5ec4f5513f62767d97a47325,France,fr,eu,image,assets/5ec4f5513f62767d97a47325/5ec4f5513f6276...,Bathroom/Toilet,larriere,19671.000000
38476,5ec4f5523f62767d97a47327,France,fr,eu,image,assets/5ec4f5523f62767d97a47327/5ec4f5523f6276...,Armchair,larriere,19671.000000
38477,5ec4f5523f62767d97a47328,France,fr,eu,image,assets/5ec4f5523f62767d97a47328/5ec4f5523f6276...,Armchair,larriere,19671.000000


In [10]:
len(data)

36754

### Split topics

In [13]:
from collections import Counter
list_topics = list(data['topics'])


separate_topics = [t.lower().strip() for topic in list_topics for t in topic.split(",") ]


print(len(separate_topics))
#print(Counter(separate_topics))
set_topics = list(set(separate_topics))
print(len(set_topics))

45692
270


In [14]:
len(separate_topics)

45692

In [15]:
# map each topic to list of corresponding images
dict_topic2img = {}
for list_topics, image_path in zip(data['topics'], data['imageRelPath']):
    for topic in list_topics.split(","):
        topic = topic.lower().strip()
        if topic not in dict_topic2img:
            dict_topic2img[topic] = set() #### here a set was used in place of list to avoid duplicate where keyword appears twice in a topic
        dict_topic2img[topic].add(image_path)

print(len(dict_topic2img))

270


In [16]:
#sanity checks 

ground_truth_counts = [len(dict_topic2img[i]) for i in dict_topic2img]
print(sum(ground_truth_counts))

45616


In [17]:
topics = dict_topic2img.keys()

### Get ground truth

In [18]:
temp = "This is a photo of "
English_topics = [temp + topic for topic in topics]

In [19]:
topic2english = {}
for topic, prompt in zip(topics, English_topics):
    topic2english[topic] = prompt

In [20]:
#topic2english

In [21]:
english_dict_topic2imgGT = {}

for topic in topics:
    key = topic2english[topic]
    english_dict_topic2imgGT[key] = dict_topic2img[topic]
print(len(english_dict_topic2imgGT))

270


In [22]:
len(english_dict_topic2imgGT['This is a photo of spices'])

419

In [23]:
import pickle
with open("list_eng2translated_text_dicts_merged.pkl", 'rb') as f:
    list_eng2translated_text_dicts_merged = pickle.load(f)

In [24]:
list_dict_trans_prompt2img = []

for lang_dict in tqdm.tqdm(list_eng2translated_text_dicts_merged):
    dict_trans_prompt2img = {}
    for eng_prompt in lang_dict.keys():
        trans = lang_dict[eng_prompt]
        dict_trans_prompt2img[trans] = english_dict_topic2imgGT[eng_prompt]
        
    list_dict_trans_prompt2img.append(dict_trans_prompt2img)

100%|██████████| 40/40 [00:00<00:00, 6018.08it/s]


In [25]:
len(list_dict_trans_prompt2img[0]['deu_Latn ; This is a photo of family snapshots ; Das ist ein Foto von Familienbildern.'])

102

In [26]:
list(list_eng2translated_text_dicts_merged[0].values())[0].split(';')[-1].strip()

'Das ist ein Foto von Familienbildern.'

In [27]:
len([t.split(';')[-1].strip() for t in list_eng2translated_text_dicts_merged[0].values()])

270

## Plotting results

In [28]:
import pickle
with open("data/lang_list_dict_topic2FNimg.pkl", 'rb') as f:
    lang_list_dict_topic2FNimg = pickle.load(f)

In [29]:
with open("data/lang_list_dict_topic2F1.pkl", 'rb') as f:
    lang_list_dict_topic2F1 = pickle.load(f)

In [65]:
with open("list_dict_lang2translation.pkl", 'rb') as f:
    list_dict_lang2translation = pickle.load(f)

In [31]:
labels=["poor", "lower-middle", "upper-middle", "rich"]

#list_of_topic2prompt_dict[1]
data["quartile"] = pd.qcut(data["income"], q=[0, 0.25, 0.5, 0.75, 1],    
                         labels=labels)

In [32]:
# map each image to corresponding topics, countries, income level ... TODO
dict_img2topic, dict_img2country, dict_img2incomelevel, dict_country2region = {}, {}, {}, {}
for list_topics, image_path, country, incomelevel, region in zip(data['topics'], data['imageRelPath'], data['country.name'], data['quartile'], data['region.id']):
    dict_img2topic[image_path] = list_topics
    dict_img2country[image_path] = country
    dict_img2incomelevel[image_path] = incomelevel
    dict_country2region[country] = region

print(len(dict_img2topic))
# print(dict_img2country)

36754


In [33]:
len(lang_list_dict_topic2FNimg[0]['deu_Latn ; This is a photo of family snapshots ; Das ist ein Foto von Familienbildern.'])

77

## Automating the analysis

In [ ]:
# all_topic_all_income_percountrydf = pd.DataFrame(columns= ['Country',
#                                                  'Forget ratio',
#                                                  'dominant_language',
#                                                  'Total images per country',
#                                                  'Forgotten images',
#                                                  'Recognized images',
#                                                  'Recall'])
# all_topic_all_country_perincomedf  = pd.DataFrame(columns = ['Income level',
#                                                  'Forget ratio',
#                                                  'Total images per income level',
#                                                  'Forgotten images',
#                                                  'dominant language',
#                                                  'Recognized images',
#                                                  'Recall'])
# language_recall_df = pd.DataFrame(columns = ['language', 'countries', 'incomelevel', 'forgotten', 'ground truth', 'recall'])


In [208]:
# all_topic_all_income_percountrydf = Forget_ratio_percountry_alltopics_df
# all_topic_all_country_perincomedf = Forget_ratio_all_country_perincome_df
# language_recall_df = Lang_recall_df

In [64]:
list_dict_lang2translation[39]

{'Swahili': ['Mufananidzo uyu unoratidza mhuri',
  'Uyu mufananidzo wemidziyo',
  'Uyu mufananidzo wemhuri',
  'Uyu mufananidzo wenzvimbo apo kudya kwemanheru',
  'Uyu mufananidzo wezvokudya',
  'Uyu mufananidzo patafura ine zvokudya',
  'Uyu mufananidzo wokutora mvura',
  'Uyu mufananidzo wechitofu / hob',
  'Uyu mufananidzo wechiedza chiri mukicheni',
  'Uyu mufananidzo wemvura yokunwa',
  'Uyu mufananidzo wetsvina yekicheni',
  'Uyu mufananidzo wokugara wega mumba mokuchera',
  'Uyu mufananidzo weimba yapashure',
  'Uyu mufananidzo pamukova wokutanga',
  'Uyu mufananidzo wemba',
  'Uyu mufananidzo tsime Cool',
  'Uyu mufananidzo wemugwagwa',
  'Uyu mufananidzo wemeno',
  'Uyu mufananidzo ruoko shure',
  'Uyu mufananidzo woruoko rwemaoko',
  'Uyu mufananidzo wemhuka',
  'Uyu mufananidzo wokurara',
  'Uyu mufananidzo wezviyo',
  'Ichi chifananidzo chemucheka',
  'Uyu mufananidzo wechigaro',
  'Uyu mufananidzo wepamberi',
  'Uyu mufananidzo wekicheni',
  'Ichi chifananidzo chemabhuku',

In [66]:
from collections import Counter

all_topic_all_income_percountrydf = pd.DataFrame()
all_topic_all_country_perincomedf = pd.DataFrame()
language_recall_df = pd.DataFrame()
for lang_dict in tqdm.tqdm(lang_list_dict_topic2FNimg):
    
    #### forgotten images block (countrywise)
    
    list_all_countriesFN = []
    list_all_incomeFN = []
    dict_topic2FNCountry = {}
    dict_all_Countries_FN = {}
    
    for topic in lang_dict:
        
        dict_lang2translation = next(item for item in list_dict_lang2translation if topic.split(';')[-1].strip() in list(item.values())[0])
        dominant_lang = list(dict_lang2translation.keys())[0]
        list_imagesFN = lang_dict[topic]
        list_countriesFN = [dict_img2country[img] for img in list_imagesFN]
        list_incomepercountry = [dict_img2incomelevel[img] for img in list_imagesFN]
        
        for country in list_countriesFN:
            list_all_countriesFN.append(country)
        for income in list_incomepercountry:
            list_all_incomeFN.append(income)
        
        dict_topic2FNCountry[topic] = Counter(list_countriesFN)

    dict_all_Countries_FN = Counter(list_all_countriesFN)
    
    
    #### ground truth images block (countrywise)
    
    list_all_countriesGT = []
    list_all_incomeGT = []
    dict_topic2GTCountry = {}
    dict_all_Countries_GT = {}
    
    dict_lang_topic2img = next(item for item in list_dict_trans_prompt2img if topic in item.keys())
    
    for topic in dict_lang_topic2img:
        list_imagesGT = dict_lang_topic2img[topic]
        list_countriesGT = [dict_img2country[img] for img in list_imagesGT]
        list_incomepercountryGT = [dict_img2incomelevel[img] for img in list_imagesGT]
        for country in list_countriesGT:
            list_all_countriesGT.append(country)
        for income in list_incomepercountryGT:
            list_all_incomeGT.append(income)
            
        dict_topic2GTCountry[topic] = Counter(list_countriesGT)

    
    dict_all_Countries_GT = Counter(list_all_countriesGT)
    
    
    
    ### forgotten images block (income level wise)
    
    list_all_incomelevelsFN = []
    dict_topic2FNIncomelevels = {}
    dict_all_incomelevels_FN = {}
    
    mapped_dict_lang_topic2FNimg = next(item for item in lang_list_dict_topic2FNimg if topic in item.keys())
    
    for topic in mapped_dict_lang_topic2FNimg:
        list_imagesFN = mapped_dict_lang_topic2FNimg[topic]
        list_incomelevelsFN = [dict_img2incomelevel[img] for img in list_imagesFN]
        for incomelevel in list_incomelevelsFN:
            list_all_incomelevelsFN.append(incomelevel)

        dict_topic2FNIncomelevels[topic] = Counter(list_incomelevelsFN)


    dict_all_incomelevels_FN = Counter(list_all_incomelevelsFN)
    
    
    
    ### ground truth image block (income level wise)
    
    ### Income level ground truth

    list_all_incomelevelsGT = []
    dict_topic2GTIncomelevels = {}
    dict_all_incomelevels_GT = {}
    
    mapped_dict_lang_topic2img = next(item for item in list_dict_trans_prompt2img if topic in item.keys())
    for topic in mapped_dict_lang_topic2img:
        list_imagesGT = mapped_dict_lang_topic2img[topic]
        list_incomelevelsGT = [dict_img2incomelevel[img] for img in list_imagesGT]
        for incomelevel in list_incomelevelsGT: 
            list_all_incomelevelsGT.append(incomelevel)
        
        dict_topic2GTIncomelevels[topic] = Counter(list_incomelevelsGT)

    
    dict_all_incomelevels_GT = Counter(list_all_incomelevelsGT)

    
    #### dataframe building
    
    ################### all_topic_all_income_percountrydf ###########
    
    Forget_ratio_allincome_percountry = Counter({key : dict_all_Countries_FN[key] / dict_all_Countries_GT[key] for key in dict_all_Countries_FN})
    Forget_ratio_allincome_percountry_df = pd.DataFrame(Forget_ratio_allincome_percountry.items(),
                        columns = ['Country', 'Forget ratio'])
    Forget_ratio_allincome_percountry_df['dominant language'] = [dominant_lang for i in list(Forget_ratio_allincome_percountry_df['Forget ratio'])]
    Forget_ratio_allincome_percountry_df['Total images per country'] = Forget_ratio_allincome_percountry_df['Country'].map(dict_all_Countries_GT)
    Forget_ratio_allincome_percountry_df['Forgotten images'] = Forget_ratio_allincome_percountry_df['Country'].map(dict_all_Countries_FN)
    Forget_ratio_allincome_percountry_df['Recognized images'] = Forget_ratio_allincome_percountry_df['Total images per country'] - Forget_ratio_allincome_percountry_df['Forgotten images']
    Forget_ratio_allincome_percountry_df['Recall'] = Forget_ratio_allincome_percountry_df['Recognized images'] / Forget_ratio_allincome_percountry_df['Total images per country']
    
    all_topic_all_income_percountrydf = pd.concat([all_topic_all_income_percountrydf, Forget_ratio_allincome_percountry_df])
    
    
    ####################### alltopic_all_country_perincome ###################
    
    Forget_ratio_all_country_perincome = Counter({key : dict_all_incomelevels_FN[key] / dict_all_incomelevels_GT[key] for key in dict_all_incomelevels_FN})
    Forget_ratio_all_country_perincome_df = pd.DataFrame(Forget_ratio_all_country_perincome.items(), columns = ['Income level', 'Forget ratio',])
    Forget_ratio_all_country_perincome_df['Total images per income level'] = Forget_ratio_all_country_perincome_df['Income level'].map(dict_all_incomelevels_GT)
    Forget_ratio_all_country_perincome_df['Forgotten images'] = Forget_ratio_all_country_perincome_df['Income level'].map(dict_all_incomelevels_FN)
    #Forget_rationperincome_allcountry_df = Forget_ratio_percountry_allincome_df
    Forget_ratio_all_country_perincome_df['dominant language'] = [dominant_lang for i in list(Forget_ratio_all_country_perincome_df['Forget ratio'])]
    Forget_ratio_all_country_perincome_df['Recognized images'] = Forget_ratio_all_country_perincome_df['Total images per income level'] - Forget_ratio_all_country_perincome_df['Forgotten images']
    Forget_ratio_all_country_perincome_df['Recall'] = Forget_ratio_all_country_perincome_df['Recognized images'] / Forget_ratio_all_country_perincome_df['Total images per income level']
    
    all_topic_all_country_perincomedf = pd.concat([all_topic_all_country_perincomedf, Forget_ratio_all_country_perincome_df])
    
    
    
    ######################## all topic, per country, per income #####################
    
    Lang_FN_df = pd.DataFrame()
    Lang_FN_df['countries'] = list_all_countriesFN
    Lang_FN_df['language'] = [dominant_lang for i in list_all_countriesFN]
    Lang_FN_df['income level'] = list_all_incomeFN
    Lang_forgotten = Lang_FN_df.groupby(['countries', 'income level']).count()
    Lang_forgotten.reset_index()
    
    lang_countries = [x[0] for x in Lang_forgotten.index]
    lang_incomelevel = [x[1] for x in Lang_forgotten.index]
    
    Lang_GT_df = pd.DataFrame()
    Lang_GT_df['countries'] = list_all_countriesGT
    Lang_GT_df['language'] = [dominant_lang for i in list_all_countriesGT]
    Lang_GT_df['income level'] = list_all_incomeGT

    
    Lang_GT = Lang_GT_df.groupby(['countries', 'income level']).count()
    Lang_GT.reset_index()
    
    Lang_recall_df = pd.DataFrame()

    Lang_recall_df['language'] = [dominant_lang for i in lang_countries]
    Lang_recall_df['countries'] = lang_countries
    Lang_recall_df['incomelevel'] = lang_incomelevel
    Lang_recall_df['forgotten'] = list(Lang_forgotten['language'])
    Lang_recall_df['ground truth'] =  list(Lang_GT['language'])
    Lang_recall_df['recall'] = 1 - (Lang_recall_df['forgotten']/ Lang_recall_df['ground truth'])
    
    language_recall_df = pd.concat([language_recall_df, Lang_recall_df])
    
    

all_topic_all_income_percountrydf.to_csv('all_topic_all_income_percountrydf')
all_topic_all_country_perincomedf.to_csv('all_topic_all_country_perincomedf')
language_recall_df.to_csv('language_recall_df')

100%|██████████| 40/40 [00:04<00:00,  8.05it/s]


## Cross analysis of performance

In [5]:
import pandas as pd
all_topic_all_income_percountrydf = pd.read_csv('all_topic_all_income_percountrydf')
all_topic_all_country_perincomedf = pd.read_csv('all_topic_all_country_perincomedf')
language_recall_df = pd.read_csv('language_recall_df')

In [68]:
language_recall_df

,Unnamed: 0,language,countries,incomelevel,forgotten,ground truth,recall
0,0,German,Austria,rich,214,284,0.246479
1,1,German,Austria,upper-middle,91,144,0.368056
2,2,German,Bangladesh,poor,557,753,0.260292
3,3,German,Bolivia,lower-middle,171,245,0.302041
4,4,German,Bolivia,poor,70,95,0.263158
...,...,...,...,...,...,...,...
5115,123,Shona,United States,rich,653,1048,0.376908
5116,124,Shona,United States,upper-middle,572,846,0.323877
5117,125,Shona,Vietnam,lower-middle,360,491,0.266802
5118,126,Shona,Vietnam,rich,94,145,0.351724


In [36]:
all_topic_all_country_perincomedf[all_topic_all_country_perincomedf['dominant language']== 'French']

,Unnamed: 0,Income level,Forget ratio,Total images per income level,Forgotten images,dominant language,Recognized images,Recall
12,0,poor,0.783344,11119,8710,French,2409,0.216656
13,1,lower-middle,0.663363,10997,7295,French,3702,0.336637
14,2,upper-middle,0.574349,11446,6574,French,4872,0.425651
15,3,rich,0.586112,12054,7065,French,4989,0.413888


In [13]:
(2409+3702)/(11119+10997)

0.27631578947368424

In [14]:
(0.216656 + 0.336637) /2

0.2766465

In [15]:
a = language_recall_df[language_recall_df['countries'] == 'Vietnam']
a[a['language']=='Vietnamese']

,Unnamed: 0,language,countries,incomelevel,forgotten,ground truth,recall
3197,125,Vietnamese,Vietnam,lower-middle,339,491,0.309572
3198,126,Vietnamese,Vietnam,rich,82,145,0.434483


In [60]:
unique_countries = list(all_topic_all_income_percountrydf['dominant language'].unique())
len(unique_countries)

39

In [69]:
## Additional dataframe for lower levels only 

low_lvl_language_recall_df = pd.DataFrame()

unique_countries = list(all_topic_all_income_percountrydf['Country'].unique())
unique_countries.sort()

for country in unique_countries:
    all_lvl = language_recall_df[language_recall_df['countries'] == country]
    unique_lvls = list(all_lvl['incomelevel'].unique())
    lvls = []
    for lvl in unique_lvls:
        if lvl in ['poor', 'lower-middle']:
            lvls.append(lvl)

    if len(lvls) == 2:

        lower_lvl_poor = all_lvl[all_lvl['incomelevel'] == 'poor']
        lower_lvl_low_mid = all_lvl[all_lvl['incomelevel'] == 'lower-middle']

        forgotten = list(lower_lvl_poor['forgotten']) + (lower_lvl_low_mid['forgotten'])
        ground_truth = list(lower_lvl_poor['ground truth']) + (lower_lvl_low_mid['ground truth'])
        recall = (ground_truth - forgotten) / ground_truth
        df = lower_lvl_poor.drop(['forgotten', 'incomelevel', 'recall', 'ground truth'], axis=1)
        df['incomelevel'] = ['poor, lower-middle' for i in df['countries']]
        df['forgotten'] = list(forgotten)
        df['ground truth'] = list(ground_truth)
        df['recall'] = list(recall)
        low_lvl_language_recall_df = pd.concat([low_lvl_language_recall_df, df])

    elif len(lvls) == 1:
        lower_lvl_poor = all_lvl[all_lvl['incomelevel'] == lvls[0]]
        df = lower_lvl_poor
        low_lvl_language_recall_df = pd.concat([low_lvl_language_recall_df, df])

    elif len(lvls) == 0:
        df = pd.DataFrame()

low_lvl_language_recall_df.to_csv('low_lvl_language_recall_df')    

In [70]:
len(low_lvl_language_recall_df[low_lvl_language_recall_df['countries'] == 'Kenya'])

40

In [90]:
len(low_lvl_language_recall_df['countries'].unique())

42

In [71]:
low_lvl_language_recall_df[low_lvl_language_recall_df['countries'] == 'Kenya']

,Unnamed: 0,language,countries,incomelevel,forgotten,ground truth,recall
52,52,German,Kenya,"poor, lower-middle",333,467,0.286938
180,52,Spanish,Kenya,"poor, lower-middle",336,467,0.280514
308,52,Portuguese,Kenya,"poor, lower-middle",328,467,0.297645
436,52,French,Kenya,"poor, lower-middle",333,467,0.286938
564,52,Chinese,Kenya,"poor, lower-middle",330,467,0.293362
692,52,Czech,Kenya,"poor, lower-middle",329,467,0.295503
820,52,Danish,Kenya,"poor, lower-middle",333,467,0.286938
948,52,Arabic,Kenya,"poor, lower-middle",344,467,0.263383
1076,52,Hindi,Kenya,"poor, lower-middle",330,467,0.293362
1204,52,Indonesian,Kenya,"poor, lower-middle",333,467,0.286938


In [7]:
all_topic_all_country_perincomedf[all_topic_all_country_perincomedf['dominant language'] == 'Arabic']

,Unnamed: 0,Income level,Forget ratio,Total images per income level,Forgotten images,dominant language,Recognized images,Recall
28,0,poor,0.790359,11119,8788,Arabic,2331,0.209641
29,1,lower-middle,0.679822,10997,7476,Arabic,3521,0.320178
30,2,upper-middle,0.615062,11446,7040,Arabic,4406,0.384938
31,3,rich,0.626597,12054,7553,Arabic,4501,0.373403


In [8]:
all_topic_all_income_percountrydf[all_topic_all_income_percountrydf['dominant language'] == 'Arabic']

,Unnamed: 0,Country,Forget ratio,dominant language,Total images per country,Forgotten images,Recognized images,Recall
441,0,Burundi,0.824773,Arabic,331,273,58,0.175227
442,1,India,0.659378,Arabic,5625,3709,1916,0.340622
443,2,Somalia,0.925926,Arabic,54,50,4,0.074074
444,3,Papua New Guinea,0.838384,Arabic,99,83,16,0.161616
445,4,Rwanda,0.707483,Arabic,294,208,86,0.292517
...,...,...,...,...,...,...,...,...
499,58,Canada,0.375000,Arabic,48,18,30,0.625000
500,59,Denmark,0.730769,Arabic,312,228,84,0.269231
501,60,Togo,0.776923,Arabic,390,303,87,0.223077
502,61,Italy,0.652344,Arabic,256,167,89,0.347656


In [9]:
specific = language_recall_df[language_recall_df['countries'] == 'Colombia']

specific[specific['language'] == 'Spanish']


,Unnamed: 0,language,countries,incomelevel,forgotten,ground truth,recall
149,21,Spanish,Colombia,lower-middle,204,322,0.366460
150,22,Spanish,Colombia,poor,251,338,0.257396
151,23,Spanish,Colombia,rich,433,721,0.399445
152,24,Spanish,Colombia,upper-middle,989,1619,0.389129


In [10]:
#all_topic_all_income_percountrydf[all_topic_all_income_percountrydf['Country'] == 'Colombia'].sort_values('Recall')

## Query for table filling

In [6]:
selected_languages = ['German', 'Spanish', 'Portuguese', 'French', 'Chinese', 'Czech', 'Danish', 'Arabic', 'Hindi', 'Indonesian', 'Farsi_Persian', 'Italian', 'Russian', 'Mongolian', 'Burmese', 'Dutch', 'Urdu', 'Romanian', 'Serbian', 'Korean', 'Swedish', 'Thai', 'Turkish', 'Ukrainian', 'Vietnamese', 'Bengali', 'Khmer', 'Oromo', 'Ewe', 'Creole', 'Swahili', 'Nepali', 'Hausa', 'Kyrgz', 'Tagalog', 'Kinyarwanda', 'Somali', 'Zulu', 'Sinhala', 'Shona']
country_lang_dict = {'Austria': 'German', 'Bangladesh': 'Bengali', 'Bolivia': 'Spanish', 'Brazil': 'Portuguese', 'Burkina Faso': 'French', 'Burundi': 'French', 'Cambodia': 'Khmer', 'Cameroon': 'French', 'Canada': 'French', 'China': 'Chinese', 'Colombia':'Spanish', "Cote d'Ivoire": 'French', 'Czech Republic': 'Czech', 'Denmark': 'Danish', 'Egypt': 'Arabic', 'Ethiopia': 'Oromo', 'France': 'French', 'Ghana': 'Ewe', 'Guatemala': 'Spanish', 'Haiti': 'Creole', 'India': 'Hindi', 'Indonesia': 'Indonesian', 'Iran': 'Farsi_Persian', 'Italy': 'Italian', 'Jordan': 'Arabic', 'Kazakhstan': 'Russian', 'Kenya': 'Swahili', 'Kyrgyzstan': 'Kyrgz', 'Lebanon': 'Arabic', 'Liberia': 'None', 'Malawi': 'None', 'Mexico': 'Spanish', 'Mongolia': 'Mongolian', 'Myanmar': 'Burmese', 'Nepal': 'Nepali', 'Netherlands': 'Dutch', 'Nigeria': 'Hausa', 'Pakistan': 'Urdu', 'Palestine': 'Arabic', 'Papua New Guinea': 'None', 'Peru': 'Spanish', 'Philippines': 'Tagalog', 'Romania': 'Romanian', 'Russia': 'Russian', 'Rwanda': 'Kinyarwanda', 'Serbia': 'Serbian', 'Somalia': 'Somali', 'South Africa': 'Zulu', 'South Korea': 'Korean', 'Spain': 'Spanish', 'Sri Lanka': 'Sinhala', 'Sweden': 'Swedish', 'Switzerland': 'German', 'Tanzania': 'Swahili', 'Thailand': 'Thai', 'Togo':'French', 'Tunisia': 'Arabic', 'Turkey': 'Turkish', 'Ukraine':'Ukrainian', 'United Kingdom': 'None', 'United States': 'Spanish', 'Vietnam': 'Vietnamese', 'Zimbabwe': 'Shona'}

In [7]:
country_lang_dict

{'Austria': 'German',
 'Bangladesh': 'Bengali',
 'Bolivia': 'Spanish',
 'Brazil': 'Portuguese',
 'Burkina Faso': 'French',
 'Burundi': 'French',
 'Cambodia': 'Khmer',
 'Cameroon': 'French',
 'Canada': 'French',
 'China': 'Chinese',
 'Colombia': 'Spanish',
 "Cote d'Ivoire": 'French',
 'Czech Republic': 'Czech',
 'Denmark': 'Danish',
 'Egypt': 'Arabic',
 'Ethiopia': 'Oromo',
 'France': 'French',
 'Ghana': 'Ewe',
 'Guatemala': 'Spanish',
 'Haiti': 'Creole',
 'India': 'Hindi',
 'Indonesia': 'Indonesian',
 'Iran': 'Farsi_Persian',
 'Italy': 'Italian',
 'Jordan': 'Arabic',
 'Kazakhstan': 'Russian',
 'Kenya': 'Swahili',
 'Kyrgyzstan': 'Kyrgz',
 'Lebanon': 'Arabic',
 'Liberia': 'None',
 'Malawi': 'None',
 'Mexico': 'Spanish',
 'Mongolia': 'Mongolian',
 'Myanmar': 'Burmese',
 'Nepal': 'Nepali',
 'Netherlands': 'Dutch',
 'Nigeria': 'Hausa',
 'Pakistan': 'Urdu',
 'Palestine': 'Arabic',
 'Papua New Guinea': 'None',
 'Peru': 'Spanish',
 'Philippines': 'Tagalog',
 'Romania': 'Romanian',
 'Russia': '

In [16]:
for i in country_lang_dict.values():
    print(i)

German
Bengali
Spanish
Portuguese
French
French
Khmer
French
French
Chinese
Spanish
French
Czech
Danish
Arabic
Oromo
French
Ewe
Spanish
Creole
Hindi
Indonesian
Farsi_Persian
Italian
Arabic
Russian
Swahili
Kyrgz
Arabic
None
None
Spanish
Mongolian
Burmese
Nepali
Dutch
Hausa
Urdu
Arabic
None
Spanish
Tagalog
Romanian
Russian
Kinyarwanda
Serbian
Somali
Zulu
Korean
Spanish
Sinhala
Swedish
German
Swahili
Thai
French
Arabic
Turkish
Ukrainian
None
None
Vietnamese
Shona


In [11]:
selected_countries = ['Austria', 'Bangladesh', 'Bolivia', 'Brazil', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'China', 'Colombia', "Cote d'Ivoire", 'Czech Republic', 'Denmark', 'Egypt', 'Ethiopia', 'France', 'Ghana', 'Guatemala', 'Haiti', 'Kenya', 'Kyrgyzstan', 'India', 'Indonesia', 'Iran', 'Italy', 'Jordan', 'Kazakhstan', 'Lebanon', 'Mexico', 'Mongolia', 'Myanmar', 'Nepal', 'Nigeria', 'Netherlands', 'Pakistan', 'Palestine', 'Peru', 'Philippines', 'Romania', 'Russia', 'Rwanda', 'Serbia', 'Somalia', 'South Africa', 'South Korea', 'Spain', 'Sri Lanka', 'Sweden', 'Switzerland', 'Tanzania', 'Thailand', 'Togo', 'Tunisia', 'Turkey', 'Ukraine', 'United States', 'Vietnam', 'Zimbabwe']

In [12]:
len(selected_countries)

59

In [13]:
import pandas as pd
low_lvl_language_recall_df = pd.read_csv('low_lvl_language_recall_df')

In [16]:
#country

In [14]:

## Query for country prompt recall for each country, low income only

language_prompt_recall_list = []
language_prompt_dict = {}
language_list = []


unique_countries = list(all_topic_all_income_percountrydf['Country'].unique())
unique_countries.sort()

df_countries = list(low_lvl_language_recall_df['countries'].unique())
df_countries.sort()


for country in unique_countries:
    if country in df_countries:
        unique_language = country_lang_dict[country]
        if unique_language != 'None':
            #print(unique_language)
            specific = low_lvl_language_recall_df[low_lvl_language_recall_df['countries'] == country]
            recall = specific[specific['language'] == unique_language]['recall']
            recall = round(recall.item(), 3)
            language_list.append(unique_language)
        else: 
            recall = '-'
            unique_language = 'None'
            language_list.append(unique_language)
        language_prompt_recall_list.append(recall)
        language_prompt_dict[country] = recall
    else: 
        recall = '-'
        unique_language = 'None'
        language_prompt_recall_list.append(recall)
        language_list.append(unique_language)
        language_prompt_dict[country] = recall

In [20]:
specific[specific['language'] == unique_language]['recall']

1679    0.14
Name: recall, dtype: float64

In [15]:
lang_countries = [country for country in selected_countries if country in df_countries]
len(lang_countries)

39

In [16]:
[country for country in df_countries if country not in lang_countries]

['Liberia', 'Malawi', 'Papua New Guinea']

In [17]:
len(language_list)

63

In [25]:
low_don_langs = [i for i in language_prompt_dict.keys() if language_prompt_dict[i]!= '-']
for i in low_don_langs:
    print(i)

Bangladesh
Bolivia
Brazil
Burkina Faso
Burundi
Cambodia
Cameroon
China
Colombia
Cote d'Ivoire
Ethiopia
Ghana
Guatemala
Haiti
India
Indonesia
Jordan
Kenya
Mongolia
Myanmar
Nepal
Nigeria
Pakistan
Palestine
Peru
Philippines
Rwanda
Serbia
Somalia
South Africa
South Korea
Tanzania
Thailand
Togo
Tunisia
Ukraine
United States
Vietnam
Zimbabwe


In [20]:
for i in language_prompt_dict.values():
    print(i)

-
0.254
0.291
0.321
0.153
0.136
0.264
0.283
-
0.297
0.311
0.204
-
-
-
0.185
-
0.204
0.33
0.168
0.278
0.369
-
-
0.395
-
0.283
-
-
-
-
-
0.206
0.232
0.278
-
0.263
0.307
0.355
-
0.23
0.312
-
-
0.241
0.312
0.074
0.181
0.412
-
-
-
-
0.291
0.254
0.246
0.292
-
0.392
-
0.438
0.31
0.14


In [21]:
specific_langs = [lang for lang in language_list if lang != 'None']

In [22]:
#language_prompt_recall_list
#len(language_prompt_recall_list)

In [23]:
Languages = ['Arabic','Burmese','Chinese','French','Hindi','Indonesian','Korean','Mongolian','Portuguese','Serbian','Spanish','Thai','Ukrainian','Urdu','Vietnamese']

In [79]:
## Query for country prompt recall for each country

language_prompt_recall_list = []
language_prompt_dict = {}
language_list = []


unique_countries = list(all_topic_all_income_percountrydf['Country'].unique())
unique_countries.sort()



for country in unique_countries:
    unique_language = country_lang_dict[country]
    if unique_language != 'None':
        specific = all_topic_all_income_percountrydf[all_topic_all_income_percountrydf['Country'] == country]
        recall = specific[specific['dominant language'] == unique_language]['Recall']
        recall = round(recall.item(), 3)
    else: 
        recall = '-'
    language_prompt_recall_list.append(recall)
    language_list.append(unique_language)
    language_prompt_dict[country] = recall
    
    
    

In [76]:
unique_language 

'Kyrgyz'

In [80]:
for i in language_prompt_recall_list:
    print(i)

0.287
0.254
0.291
0.351
0.153
0.136
0.274
0.32
0.667
0.314
0.374
0.204
0.336
0.253
0.468
0.207
0.429
0.204
0.33
0.168
0.364
0.384
0.398
0.43
0.396
0.535
0.321
0.471
0.469
-
-
0.477
0.206
0.232
0.303
0.413
0.299
0.382
0.355
-
0.23
0.33
0.43
0.395
0.241
0.353
0.074
0.231
0.416
0.389
0.515
0.241
0.296
0.322
0.28
0.246
0.292
0.325
0.384
-
-
0.338
0.14


In [26]:
# unique_languages = list(all_topic_all_income_percountrydf['dominant language'].unique())
# unique_languages.sort()
# unique_languages
low_lvl_language_recall_df

,Unnamed: 0.1,Unnamed: 0,language,countries,incomelevel,forgotten,ground truth,recall
0,2,2,German,Bangladesh,poor,557,753,0.260292
1,130,2,Spanish,Bangladesh,poor,567,753,0.247012
2,258,2,Portuguese,Bangladesh,poor,563,753,0.252324
3,386,2,French,Bangladesh,poor,555,753,0.262948
4,514,2,Chinese,Bangladesh,poor,561,753,0.254980
...,...,...,...,...,...,...,...,...
1045,2687,127,Swedish,Zimbabwe,poor,387,450,0.140000
1046,2815,127,Thai,Zimbabwe,poor,380,450,0.155556
1047,2943,127,Turkish,Zimbabwe,poor,386,450,0.142222
1048,3071,127,Ukrainian,Zimbabwe,poor,391,450,0.131111


In [27]:
all_topic_all_income_percountrydf

,Unnamed: 0,Country,Forget ratio,dominant language,Total images per country,Forgotten images,Recognized images,Recall
0,0,Burundi,0.836858,German,331,277,54,0.163142
1,1,India,0.654578,German,5625,3682,1943,0.345422
2,2,Somalia,0.907407,German,54,49,5,0.092593
3,3,Papua New Guinea,0.838384,German,99,83,16,0.161616
4,4,Rwanda,0.687075,German,294,202,92,0.312925
...,...,...,...,...,...,...,...,...
1570,58,Sweden,0.741071,Vietnamese,224,166,58,0.258929
1571,59,Canada,0.375000,Vietnamese,48,18,30,0.625000
1572,60,Denmark,0.740385,Vietnamese,312,231,81,0.259615
1573,61,Togo,0.764103,Vietnamese,390,298,92,0.235897


In [37]:

## Query for country prompt recall for each country for low income only 

lang_prompt_top_dict = {}
lang_prompt_worst_dict = {}
list_best_prompt = []
list_worst_prompt = []
bestest_prompt = []
rank_list = []

unique_countries = list(all_topic_all_income_percountrydf['Country'].unique())
unique_countries.sort()

for country in unique_countries:
    if country  in lang_countries:
        
        #recall = specific[specific == unique_language]['Recall']
        specific = low_lvl_language_recall_df[low_lvl_language_recall_df['countries'] == country]
        unique_language = country_lang_dict[country]
        if unique_language != 'None':
            rank = list(specific.sort_values('recall', ascending = False)['language']).index(unique_language)
            rank = rank +1
        else: 
            rank = '-'

        best_lang_promt = list(specific.sort_values('recall', ascending = False)['language'][:5])
        best_lang_recall = list(round(specific.sort_values('recall', ascending = False)['recall'][:1], 3))
        worst_lang_prompt = list(specific.sort_values('recall', ascending = False)['language'][-5:])
        list_best_prompt.append(best_lang_promt)
        list_worst_prompt.append(worst_lang_prompt[::-1])
        rank_list.append(rank)

        bestest_lang_prompt = best_lang_promt[0], best_lang_recall
        bestest_prompt.append(bestest_lang_prompt) 
        lang_prompt_top_dict[country] = best_lang_promt

        lang_prompt_worst_dict[country] = worst_lang_prompt[::-1]
        
    else:
        list_best_prompt.append(['-'])
        list_worst_prompt.append(['-'])
        rank_list.append('-')
        
        best_lang_prompt = '-'
        #bestest_lang_prompt = '-', '-'
        bestest_prompt.append('-') 
        lang_prompt_top_dict[country] = '-'

        lang_prompt_worst_dict[country] = '-'
        
        

In [41]:
for i in bestest_prompt:
    print(i)

-
('Kyrgz', [0.268])
('Indonesian', [0.338])
('Hindi', [0.328])
('Kyrgz', [0.166])
('Burmese', [0.193])
('Swahili', [0.38])
('Spanish', [0.285])
-
('Indonesian', [0.31])
('French', [0.329])
('Swahili', [0.238])
-
-
-
('Urdu', [0.318])
-
('German', [0.362])
('Indonesian', [0.348])
('Chinese', [0.201])
('Indonesian', [0.28])
('Indonesian', [0.369])
-
-
('Indonesian', [0.443])
-
('Portuguese', [0.298])
-
-
-
-
-
('Danish', [0.33])
('Indonesian', [0.257])
('Hindi', [0.295])
-
('German', [0.28])
('Urdu', [0.307])
('Portuguese', [0.42])
-
('Portuguese', [0.248])
('Swahili', [0.317])
-
-
('Hausa', [0.33])
('Hindi', [0.369])
('Kyrgz', [0.167])
('Chinese', [0.233])
('Indonesian', [0.475])
-
-
-
-
('Indonesian', [0.294])
('Tagalog', [0.342])
('Hindi', [0.267])
('Italian', [0.314])
-
('German', [0.454])
-
('Vietnamese', [0.475])
('Indonesian', [0.34])
('Farsi_Persian', [0.178])


In [35]:
## Query for country prompt recall for each country

lang_prompt_top_dict = {}
lang_prompt_worst_dict = {}
list_best_prompt = []
list_worst_prompt = []
bestest_prompt = []
rank_list = []

unique_countries = list(all_topic_all_income_percountrydf['Country'].unique())
unique_countries.sort()

for country in unique_countries:
    
    #recall = specific[specific == unique_language]['Recall']
    specific = all_topic_all_income_percountrydf[all_topic_all_income_percountrydf['Country'] == country]
    unique_language = country_lang_dict[country]
    if unique_language != 'None':
        rank = list(specific.sort_values('Recall', ascending = False)['dominant language']).index(unique_language)
        rank = rank +1
    else: 
        rank = '-'
        
    best_lang_promt = list(specific.sort_values('Recall', ascending = False)['dominant language'][:5])
    best_lang_recall = list(round(specific.sort_values('Recall', ascending = False)['Recall'][:1], 3))
    worst_lang_prompt = list(specific.sort_values('Recall', ascending = False)['dominant language'][-5:])
    list_best_prompt.append(best_lang_promt)
    list_worst_prompt.append(worst_lang_prompt[::-1])
    rank_list.append(rank)
    
    bestest_lang_prompt = best_lang_promt[0], best_lang_recall
    bestest_prompt.append(bestest_lang_prompt) 
    lang_prompt_top_dict[country] = best_lang_promt
    
    lang_prompt_worst_dict[country] = worst_lang_prompt[::-1]

In [36]:
bestest_prompt

[('French', [0.32]),
 ('Kyrgz', [0.268]),
 ('Indonesian', [0.338]),
 ('Hindi', [0.352]),
 ('Kyrgz', [0.166]),
 ('Burmese', [0.193]),
 ('Tagalog', [0.391]),
 ('Indonesian', [0.332]),
 ('Portuguese', [0.708]),
 ('Indonesian', [0.339]),
 ('Portuguese', [0.379]),
 ('Swahili', [0.238]),
 ('Hindi', [0.389]),
 ('Dutch', [0.279]),
 ('Indonesian', [0.535]),
 ('Danish', [0.365]),
 ('French', [0.429]),
 ('German', [0.362]),
 ('Indonesian', [0.348]),
 ('Chinese', [0.201]),
 ('Indonesian', [0.374]),
 ('Indonesian', [0.384]),
 ('Indonesian', [0.446]),
 ('French', [0.438]),
 ('Swahili', [0.437]),
 ('Dutch', [0.594]),
 ('Portuguese', [0.336]),
 ('Swahili', [0.635]),
 ('Swahili', [0.506]),
 ('Burmese', [0.28]),
 ('Shona', [0.156]),
 ('Italian', [0.494]),
 ('Danish', [0.33]),
 ('Indonesian', [0.257]),
 ('Hindi', [0.317]),
 ('French', [0.443]),
 ('German', [0.324]),
 ('Spanish', [0.396]),
 ('Portuguese', [0.42]),
 ('Oromo', [0.232]),
 ('Portuguese', [0.248]),
 ('Swahili', [0.34]),
 ('Spanish', [0.468]),


In [32]:
##### recall_incomelevel_list = []
country_income_level_country_prompt_recall_dict = {}

for country in unique_countries:
    unique_language = country_lang_dict[country]
    if unique_language != 'None':
        all_levels = language_recall_df[language_recall_df['countries'] == country]
        all_levels_specific = all_levels[all_levels['language'] == unique_language]
        incm_lst = ['poor', 'lower-middle', 'upper-middle', 'rich']
        recallforincm = []
        for level in incm_lst:
            if level in list(all_levels_specific['incomelevel']):
                recall_val = round(all_levels_specific[all_levels_specific['incomelevel'] == level]['recall'].item(), 3)
                recallforincm.append(recall_val)
            else: 
                recall_val = '-'
                recallforincm.append(recall_val)
    else:
        recallforincm = ['-', '-', '-', '-']
    country_income_level_country_prompt_recall_dict[country] = recallforincm

country_income_level_country_prompt_recall_dict

{'Austria': ['-', '-', 0.368, 0.246],
 'Bangladesh': ['-', '-', '-', '-'],
 'Bolivia': [0.295, 0.29, '-', '-'],
 'Brazil': ['-', 0.321, 0.392, 0.322],
 'Burkina Faso': [0.153, '-', '-', '-'],
 'Burundi': [0.136, '-', '-', '-'],
 'Cambodia': ['-', '-', '-', '-'],
 'Cameroon': [0.152, 0.348, 0.436, '-'],
 'Canada': ['-', '-', '-', 0.667],
 'China': [0.211, 0.308, 0.352, 0.28],
 'Colombia': [0.257, 0.366, 0.389, 0.399],
 "Cote d'Ivoire": [0.204, '-', '-', '-'],
 'Czech Republic': ['-', '-', '-', 0.336],
 'Denmark': ['-', '-', '-', 0.253],
 'Egypt': ['-', '-', 0.468, '-'],
 'Ethiopia': ['-', '-', '-', '-'],
 'France': ['-', '-', 0.464, 0.406],
 'Ghana': ['-', '-', '-', '-'],
 'Guatemala': ['-', 0.33, '-', '-'],
 'Haiti': [0.197, '-', '-', '-'],
 'India': [0.143, 0.363, 0.405, 0.412],
 'Indonesia': [0.323, 0.386, 0.429, 0.598],
 'Iran': ['-', '-', 0.423, 0.367],
 'Italy': ['-', '-', '-', 0.43],
 'Jordan': ['-', 0.395, '-', 0.398],
 'Kazakhstan': ['-', '-', 0.535, '-'],
 'Kenya': ['-', '-', 

In [33]:
for i in country_income_level_country_prompt_recall_dict.values():
    print(i[3])

0.246
-
-
0.322
-
-
-
-
0.667
0.28
0.399
-
0.336
0.253
-
-
0.406
-
-
-
0.412
0.598
0.367
0.43
0.398
-
-
-
-
-
-
0.464
-
-
-
0.43
-
0.364
-
-
-
-
0.43
0.492
-
0.381
-
-
0.463
0.389
-
0.273
0.296
-
-
-
-
0.325
0.382
-
-
0.434
-


In [12]:
## Query for overall recall across countries and topics for each country-suffix prompt

overall_recall_per_prompt = {}

unique_languages = list(all_topic_all_income_percountrydf['dominant language'].unique())
unique_languages.sort()
# unique_languages

for lang in unique_languages:
    sub_df = all_topic_all_country_perincomedf[all_topic_all_country_perincomedf['dominant language'] == lang]
    incm_lst = ['poor', 'lower-middle', 'upper-middle', 'rich']
    orecallforincm = []
    for level in incm_lst:
        orecall_val = round(sub_df[sub_df['Income level'] == level]['Recall'].item(), 3)
        orecallforincm.append(orecall_val)
    overall_recall_per_prompt[lang] = orecallforincm

overall_recall_per_prompt

{'Arabic': [0.21, 0.32, 0.385, 0.373],
 'Bengali': [0.209, 0.33, 0.407, 0.389],
 'Burmese': [0.215, 0.304, 0.36, 0.342],
 'Chinese': [0.215, 0.325, 0.391, 0.373],
 'Creole': [0.21, 0.326, 0.401, 0.38],
 'Czech': [0.199, 0.323, 0.403, 0.388],
 'Danish': [0.208, 0.337, 0.417, 0.4],
 'Dutch': [0.211, 0.336, 0.425, 0.408],
 'Ewe': [0.147, 0.193, 0.221, 0.206],
 'Farsi_Persian': [0.219, 0.318, 0.391, 0.381],
 'French': [0.217, 0.337, 0.426, 0.414],
 'German': [0.215, 0.331, 0.41, 0.39],
 'Hausa': [0.206, 0.316, 0.384, 0.364],
 'Hindi': [0.222, 0.345, 0.418, 0.4],
 'Indonesian': [0.221, 0.348, 0.424, 0.405],
 'Italian': [0.211, 0.343, 0.427, 0.414],
 'Khmer': [0.164, 0.22, 0.248, 0.232],
 'Kinyarwanda': [0.167, 0.237, 0.288, 0.272],
 'Korean': [0.204, 0.322, 0.402, 0.379],
 'Kyrgz': [0.216, 0.309, 0.367, 0.357],
 'Mongolian': [0.137, 0.209, 0.25, 0.234],
 'Nepali': [0.207, 0.325, 0.409, 0.397],
 'Oromo': [0.158, 0.209, 0.247, 0.234],
 'Portuguese': [0.213, 0.34, 0.426, 0.412],
 'Romanian': [

In [86]:
for i in overall_recall_per_prompt.values():
    print(i[3])

0.373
0.389
0.342
0.373
0.38
0.388
0.4
0.408
0.206
0.381
0.414
0.39
0.364
0.4
0.405
0.414
0.232
0.272
0.379
0.357
0.234
0.397
0.234
0.412
0.389
0.399
0.356
0.305
0.357
0.314
0.409
0.389
0.386
0.375
0.336
0.385
0.387
0.391
0.395
0.334


In [15]:
print(all_lang)

['Arabic', 'Bengali', 'Burmese', 'Chinese', 'Creole', 'Czech', 'Danish', 'Dutch', 'Ewe', 'Farsi_Persian', 'French', 'German', 'Hausa', 'Hindi', 'Indonesian', 'Italian', 'Khmer', 'Kinyarwanda', 'Korean', 'Kyrgz', 'Mongolian', 'Nepali', 'Oromo', 'Portuguese', 'Romanian', 'Russian', 'Serbian', 'Shona', 'Sinhala', 'Somali', 'Spanish', 'Swahili', 'Swedish', 'Tagalog', 'Thai', 'Turkish', 'Ukrainian', 'Urdu', 'Vietnamese', 'Zulu']


In [13]:
all_lang = [i for i in overall_recall_per_prompt.keys()]
all_lang_poor = [i[0] for i in overall_recall_per_prompt.values()]
all_lang_low_mid = [i[1] for i in overall_recall_per_prompt.values()]
all_lang_up_mid = [i[2] for i in overall_recall_per_prompt.values()]
all_lang_rich = [i[3] for i in overall_recall_per_prompt.values()]

In [ ]:
all_lang_poor = np.array(all_lang_poor)
all_lang_poor = np.array(all_lang_poor)
all_lang_poor = np.array(all_lang_poor)
all_lang_poor = np.array(all_lang_poor)

In [ ]:
np.save('dom_lang_poor_drop.npy', poor)

In [34]:
## Query for overall recall across countries and topics for each country-suffix prompt

overall_recall_per_prompt = {}

unique_languages = list(all_topic_all_income_percountrydf['dominant language'].unique())
unique_languages.sort()
# unique_languages

for lang in Languages:
    sub_df = all_topic_all_country_perincomedf[all_topic_all_country_perincomedf['dominant language'] == lang]
    incm_lst = ['poor', 'lower-middle', 'upper-middle', 'rich']
    orecallforincm = []
    for level in incm_lst:
        orecall_val = round(sub_df[sub_df['Income level'] == level]['Recall'].item(), 3)
        orecallforincm.append(orecall_val)
    overall_recall_per_prompt[lang] = orecallforincm

overall_recall_per_prompt

{'Arabic': [0.21, 0.32, 0.385, 0.373],
 'Burmese': [0.215, 0.304, 0.36, 0.342],
 'Chinese': [0.215, 0.325, 0.391, 0.373],
 'French': [0.217, 0.337, 0.426, 0.414],
 'Hindi': [0.222, 0.345, 0.418, 0.4],
 'Indonesian': [0.221, 0.348, 0.424, 0.405],
 'Korean': [0.204, 0.322, 0.402, 0.379],
 'Mongolian': [0.137, 0.209, 0.25, 0.234],
 'Portuguese': [0.213, 0.34, 0.426, 0.412],
 'Serbian': [0.192, 0.308, 0.372, 0.356],
 'Spanish': [0.207, 0.338, 0.425, 0.409],
 'Thai': [0.197, 0.297, 0.349, 0.336],
 'Ukrainian': [0.207, 0.33, 0.407, 0.387],
 'Urdu': [0.215, 0.33, 0.406, 0.391],
 'Vietnamese': [0.206, 0.328, 0.411, 0.395]}

In [27]:
for i in overall_recall_per_prompt.values():
    print(i[0])

0.21
0.215
0.215
0.199
0.208
0.211
0.219
0.217
0.215
0.222
0.221
0.211
0.204
0.137
0.213
0.203
0.211
0.192
0.207
0.205
0.197
0.205
0.207
0.215
0.206


In [35]:
poor = [round(i[0] - 0.224, 2) for i in overall_recall_per_prompt.values()]
low_mid = [round(i[1] - 0.359 , 2) for i in overall_recall_per_prompt.values()]
up_mid = [round(i[2] - 0.456, 2) for i in overall_recall_per_prompt.values()]
rich = [round(i[3] - 0.436, 2) for i in overall_recall_per_prompt.values()]
#poor

In [36]:
poor = np.array(poor)
low_mid = np.array(low_mid)
up_mid = np.array(up_mid)
rich = np.array(rich)

In [38]:
np.save('dom_lang_poor_drop.npy', poor)
np.save('dom_lang_low_mid_drop.npy', low_mid)
np.save('dom_lang_up_mid_drop.npy', up_mid)
np.save('dom_lang_rich_drop.npy', rich)

In [34]:
all_lang = [i for i in overall_recall_per_prompt.keys()]
all_lang

['Arabic',
 'Burmese',
 'Chinese',
 'Czech',
 'Danish',
 'Dutch',
 'Farsi_Persian',
 'French',
 'German',
 'Hindi',
 'Indonesian',
 'Italian',
 'Korean',
 'Mongolian',
 'Portuguese',
 'Romanian',
 'Russian',
 'Serbian',
 'Spanish',
 'Swedish',
 'Thai',
 'Turkish',
 'Ukrainian',
 'Urdu',
 'Vietnamese']

In [ ]:
len()

In [37]:
poor

array([-0.01, -0.01, -0.01, -0.02, -0.02, -0.01, -0.01, -0.01, -0.01,
       -0.  , -0.  , -0.01, -0.02, -0.09, -0.01, -0.02, -0.01, -0.03,
       -0.02, -0.02, -0.03, -0.02, -0.02, -0.01, -0.02])

In [87]:
## all lower income per country

participating_countries = list(low_lvl_language_recall_df['countries'].unique())

In [88]:
len(participating_countries)

42

In [26]:
## dataframe for heatmap

specific_langs = [lang for lang in language_list if lang != 'None']
specific_langs.sort(reverse=False)
set_specific_langs = set(specific_langs)
list_langs= list(set_specific_langs)
list_langs.sort()
#print(set_specific_langs)

In [27]:
list_langs

['Arabic',
 'Bengali',
 'Burmese',
 'Chinese',
 'Creole',
 'Ewe',
 'French',
 'Hausa',
 'Hindi',
 'Indonesian',
 'Khmer',
 'Kinyarwanda',
 'Korean',
 'Mongolian',
 'Nepali',
 'Oromo',
 'Portuguese',
 'Serbian',
 'Shona',
 'Somali',
 'Spanish',
 'Swahili',
 'Tagalog',
 'Thai',
 'Ukrainian',
 'Urdu',
 'Vietnamese',
 'Zulu']

In [28]:
len(lang_countries)

39

In [29]:
lang2recallbycountry = {}
list_oflists = []
for lang in list_langs:
    lang_recall = []
    for country in lang_countries:
        specific = low_lvl_language_recall_df[low_lvl_language_recall_df['countries'] == country]
        recall = round(specific[specific['language'] == lang]['recall'].item(), 3)
        lang_recall.append(recall)
        
    lang2recallbycountry[lang] = lang_recall
    list_oflists.append(lang_recall)

In [30]:
lang2recallbycountry['Countries'] = lang_countries
    

In [31]:
len(list_oflists)

28

In [32]:
import numpy as np

maplist = np.array(list_oflists)
np.save('maplist.npy', maplist)

In [33]:
lang2recallbycountrydf = pd.DataFrame.from_dict(lang2recallbycountry)
lang_recallbycountrydf=lang2recallbycountrydf.set_index('Countries')
lang_recallbycountrydf

,Arabic,Bengali,Burmese,Chinese,Creole,Ewe,French,Hausa,Hindi,Indonesian,...,Shona,Somali,Spanish,Swahili,Tagalog,Thai,Ukrainian,Urdu,Vietnamese,Zulu
Countries,,,,,,,,,,,,,,,,,,,,,
Bangladesh,0.254,0.254,0.246,0.255,0.239,0.169,0.263,0.247,0.251,0.247,...,0.222,0.210,0.247,0.244,0.259,0.240,0.250,0.262,0.231,0.232
Bolivia,0.300,0.285,0.282,0.309,0.326,0.165,0.324,0.282,0.332,0.338,...,0.279,0.259,0.291,0.326,0.294,0.268,0.326,0.315,0.312,0.306
Brazil,0.303,0.310,0.287,0.298,0.307,0.154,0.320,0.287,0.328,0.324,...,0.247,0.262,0.315,0.310,0.303,0.275,0.314,0.316,0.317,0.272
Burkina Faso,0.126,0.139,0.161,0.143,0.126,0.065,0.153,0.141,0.139,0.129,...,0.119,0.144,0.122,0.151,0.149,0.141,0.131,0.149,0.126,0.124
Burundi,0.175,0.157,0.193,0.142,0.160,0.136,0.136,0.151,0.166,0.163,...,0.169,0.154,0.154,0.175,0.157,0.154,0.151,0.175,0.166,0.166
Cambodia,0.336,0.357,0.344,0.342,0.350,0.236,0.369,0.338,0.365,0.365,...,0.301,0.319,0.364,0.380,0.377,0.323,0.341,0.352,0.354,0.342
Cameroon,0.264,0.267,0.262,0.275,0.260,0.172,0.283,0.242,0.269,0.281,...,0.219,0.231,0.285,0.264,0.269,0.240,0.262,0.258,0.262,0.219
China,0.290,0.292,0.274,0.297,0.279,0.184,0.279,0.285,0.303,0.310,...,0.233,0.251,0.290,0.297,0.289,0.264,0.284,0.281,0.276,0.254
Colombia,0.277,0.314,0.291,0.305,0.298,0.191,0.329,0.276,0.300,0.317,...,0.241,0.245,0.311,0.306,0.306,0.253,0.292,0.295,0.292,0.274


In [34]:
lang_recallbycountrydf.to_csv('lang2recallbycountrydf.csv')